### Note.

* multiclass (y가 분류) : [1, 0, 0]
* multilabel (y의 값이 여러개) : [1, 0, 1]
* multitask (output이 여러개) : [1, 0, 0], [0, 1, 0, 0]

* 다분류시에는 차원을 늘려줌 (one-hot encoding)
 -> output 노드의 개수는 차원 개수와 동일하게 함
* 전체 확률의 합을 1로 만들어주는 함수 __softmax__
  * 그래서 확률로 계산해주는구나,,

* feature extractor

In [1]:
test1 = 2 / 10000

In [3]:
test2 = (1 + 1/10000) - (1 - 1/10000)

In [4]:
test1

0.0002

In [5]:
test2

0.00019999999999997797

In [ ]:
# onehot-encoding 된 y-> sparse_categorical_crossentropy
# 정수형태 카테고리인 y -> categorical_crossentropy

---

In [11]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [32]:
model = Sequential([
    Dense(25, activation='relu', name='layer1'),
    Dense(15, activation='relu', name='layer2'),
    Dense(10, activation='linear', name='output')
])

In [15]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [16]:
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True))

---

In [102]:
import numpy as np
import tensorflow as tf
tfk = tf.keras
tfkl = tfk.layers

In [115]:
input = tfk.Input(2, name='input_layer')
a1 = tfkl.Dense(25, 'relu', name='layer1')(input)
a2 = tfkl.Dense(15, 'relu', name='layer2')(a1)
output = tfkl.Dense(10, 'relu', name='output')(a2)

In [116]:
model = tfk.Model(input, output, name='my_model')

In [117]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 2)]               0         
                                                                 
 layer1 (Dense)              (None, 25)                75        
                                                                 
 layer2 (Dense)              (None, 15)                390       
                                                                 
 output (Dense)              (None, 10)                160       
                                                                 
Total params: 625
Trainable params: 625
Non-trainable params: 0
_________________________________________________________________


In [118]:
n = 20000
x = np.random.randn(n, 2)
y = np.random.randint(0, 9, size=n)

In [119]:
x.shape, y.shape

((20000, 2), (20000,))

In [120]:
tf.one_hot(y, 10, dtype=tf.int32) # np.eye(10)[y].astype('int')

<tf.Tensor: shape=(20000, 10), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [121]:
y_onehot = np.eye(10)[y]

In [122]:
y_onehot.shape

(20000, 10)

In [125]:
model.compile(loss=tfk.losses.CategoricalCrossentropy(from_logits=True))

In [126]:
model.fit(x,y_onehot, epochs=10)

Epoch 1/10
625/625 [==============================] - 2s 2ms/step - loss: 2.2969
Epoch 2/10
625/625 [==============================] - 1s 2ms/step - loss: 2.2910
Epoch 3/10
625/625 [==============================] - 1s 2ms/step - loss: 2.2825
Epoch 4/10
625/625 [==============================] - 1s 2ms/step - loss: 2.2754
Epoch 5/10
625/625 [==============================] - 1s 2ms/step - loss: 2.2744
Epoch 6/10
625/625 [==============================] - 1s 2ms/step - loss: 2.2736
Epoch 7/10
625/625 [==============================] - 1s 2ms/step - loss: 2.2741
Epoch 8/10
625/625 [==============================] - 1s 2ms/step - loss: 2.2738
Epoch 9/10
625/625 [==============================] - 1s 2ms/step - loss: 2.2735
Epoch 10/10
625/625 [==============================] - 1s 2ms/step - loss: 2.2738


In [127]:
model.evaluate(x)

625/625 [==============================] - 1s 2ms/step - loss: 0.0000e+00


0.0

In [128]:
y_hat = model.predict(x)

625/625 [==============================] - 1s 2ms/step


In [134]:
y_hat.sum(axis=-1)

array([4.897428, 6.292034, 5.577613, ..., 6.971848, 5.62802 , 5.328928],
      dtype=float32)

In [135]:
f_x = tf.nn.softmax(y_hat)

In [139]:
tf.reduce_sum(f_x, axis=-1) # f_x.numpy().sum(axis=-1)

<tf.Tensor: shape=(20000,), dtype=float32, numpy=
array([0.9999999, 1.       , 1.       , ..., 1.0000001, 1.       ,
       0.9999999], dtype=float32)>

In [142]:
f_x.numpy().sum(axis=-1).round()

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)